In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\LightHead_Processed_Files
data = pd.read_csv('LightHeadCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\LightHead_Processed_Files


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_LightHead,cg17960051,cg06562372,cg04099454,cg18542829,...,cg08634012,cg09543783,cg05227434,cg08939489,cg26857677,cg20313722,cg11696249,cg07239961,cg07420027,cg08819883
0,200991620021_R01C01,3001,2,1,1,0,0.019716,0.024434,0.895146,0.024184,...,0.258948,0.805663,0.03661,0.036753,0.256459,0.750087,0.110203,0.822496,0.701242,0.82276


In [3]:
X = data.drop(['PATNO','Sentrix','HYS','Gender','MDSP_LightHead'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7391304347826086

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100


BorutaPy finished running.

Iteration: 	38 / 100
Confirmed: 	2
Tentative: 	0
Rejected: 	7039


BorutaPy(estimator=RandomForestClassifier(n_estimators=20,
                                          random_state=RandomState(MT19937) at 0x25271112C40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x25271112C40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head()

,CpG,Ranking
0,APPRDX,1
1172,cg00356916,1
6193,cg12220605,2
1594,cg17422176,3
6432,cg12683454,4


In [10]:
Top50.to_csv ('LightHead_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('LightHead_Boruta_All.csv', index = False)